In [1]:
from uuid import uuid4
import numpy as np
from numpy.typing import NDArray
import numba as nb
from scipy import stats
from typing import Callable, Tuple

In [2]:
class EventType:
    ARRIVAL = 'Arrival'
    LEAVE = 'Leave'
    DEPARTURE = 'Departure'

In [3]:
def solver_iteration(
    T: float,
    arrive_cdf: Callable[[], float],
    departure_cdf: Callable[[], float],
    leave_cdf: Callable[[], float],
    n: int,
    N: int
):
    leave_num = 0
    response_times = np.array([])
    
    arrival_time = arrive_cdf()
    leave = leave_cdf()
    
    request_times_queue = np.empty((2, 0), dtype=float)
    departure_times = np.full((2, n), -1, dtype=float)

    time = 0.0
    while time < T:
        time, event_type = get_next_event(arrival_time, request_times_queue, departure_times)            
        
        if event_type == EventType.ARRIVAL:
            arrival_time, leaves, request_times_queue, departure_times = arrival(
                time,
                request_times_queue,
                departure_times,
                arrive_cdf,
                departure_cdf,
                leave_cdf,
                N
            )
            if leaves:
                leave_num += 1
        elif event_type == EventType.DEPARTURE:
            response_time, request_times_queue, departure_time = departure(
                time,
                request_times_queue,
                departure_times,
                departure_cdf
            )
            response_times = np.append(response_times, response_time)
        else:
            request_times_queue = on_leave(time, request_times_queue)
            leave_num += 1
            
    return np.mean(response_times), leave_num

In [26]:
def get_next_event(
    arrival_time: float,
    request_times_queue: NDArray[Tuple[float, float]],
    departure_times: NDArray[Tuple[float, float]],
) -> Tuple[float, str]:
    event_type = EventType.ARRIVAL
    min_time = arrival_time

    min_departure_time = np.min(departure_times[1][departure_times[1] != -1], initial=min_time)
    min_leave_time = np.min(request_times_queue[1], initial=min_time)
    
    if min_departure_time < min_time: 
        min_time = min_departure_time
        event_type = EventType.DEPARTURE
    if min_leave_time < min_time:
        min_time = min_leave_time
        event_type = EventType.LEAVE

    return min_time, event_type

def arrival(
    time: float,
    request_times_queue: NDArray[Tuple[float, float]],
    departure_times: NDArray[Tuple[float, float]],
    arrive_cdf: Callable[[], float],
    departure_cdf: Callable[[], float],
    leave_cdf: Callable[[], float],
    N: int
) -> Tuple[float, bool, NDArray[Tuple[float, float]], NDArray[Tuple[float, float]]]:
    all_servers_busy = True
    leaves = False
    
    for i in range (len(departure_times[1])):
        if departure_times[1][i] == -1: # если сервер свободен
            all_servers_busy = False

            mu = departure_cdf()
            departure_times[0][i] = time
            departure_times[1][i] = time + mu # планируем обработку запроса
            break

    if all_servers_busy:
        gamma = leave_cdf()
        if len(request_times_queue[1]) == N:
            leaves = True
        else:
            request_times_queue = np.append(request_times_queue, [[time], [time + gamma]], axis=1)
    
    alpha = arrive_cdf()
    return time + alpha, leaves, request_times_queue, departure_times

def departure(
    time: float,
    request_times_queue: NDArray[Tuple[float, float]],
    departure_times: NDArray[Tuple[float, float]],
    departure_cdf: Callable[[], float],
) -> Tuple[float, NDArray[Tuple[float, float]], NDArray[Tuple[float, float]]]:
    i = np.where(departure_times[1] == time)[0]
    response_time = departure_times[1][i] - departure_times[0][i]
    
    if (len(request_times_queue[1]) == 0): # если очередь пуста
        departure_times[0, i] = -1
        departure_times[1, i] = -1
    else:
        departure_times[0, i] = request_times_queue[[0], [0]]
        
        gamma = departure_cdf()
        departure_times[1, i] = time + gamma # планируем обработку запроса
        request_times_queue = np.delete(request_times_queue, [[0], [0]], axis=1)
    return response_time, request_times_queue, departure_times

def on_leave(
    time: float,
    request_times_queue: NDArray[Tuple[float, float]]
) -> NDArray[Tuple[float, float]]:
    i = np.where(request_times_queue[1] == time)[0]
    request_times_queue = np.delete(request_times_queue, [[i], [i]], axis=1)

    return request_times_queue

In [88]:
arrive_cdf = lambda: stats.expon.rvs(0, 0.3)
departure_cdf = lambda: stats.expon.rvs(0, 150)
leave_cdf = lambda: stats.expon.rvs(0, 500)

In [89]:
solver_iteration(100000, arrive_cdf, departure_cdf, leave_cdf, 1, 50000)

(3143.557391066418, 331370)